In [5]:
import os
import gzip
import csv
import pandas as pd
import re
import numpy as np
import tarfile

In [ ]:
# Reference: https://www.kaggle.com/datasets/noaa/noaa-global-surface-summary-of-the-day/code?datasetId=55784&sortBy=voteCount
# Function to preprocess station file content
def process_content(data):
    # Extracting and formatting data headers
    headers = data.pop(0)
    headers = [headers[ind] for ind in [0, 1, 2, 3, 4, 8, 11, 12, 13]]

    # Loop through data rows and restructure the content
    for d in range(len(data)):
        data[d] = [data[d][ind] for ind in [0, 1, 2, 3, 5, 13, 17, 18, 19]]
    data = pd.DataFrame(data, columns=headers)

    # Renaming columns and cleaning up specific columns
    data.rename(columns={'STN---': 'USAF'}, inplace=True)
    data['MAX'] = data['MAX'].apply(lambda x: re.sub(r"\*$", "", x))
    data['MIN'] = data['MIN'].apply(lambda x: re.sub(r"\*$", "", x))
    data['PRCP'] = data['PRCP'].apply(lambda x: re.sub(x, x[:-1], x))

    # Converting certain columns to numeric and date formats
    data[['WBAN', 'TEMP', 'DEWP', 'WDSP', 'MAX', 'MIN', 'PRCP']] = data[
        ['WBAN', 'TEMP', 'DEWP', 'WDSP', 'MAX', 'MIN', 'PRCP']
    ].apply(pd.to_numeric)
    data['YEARMODA'] = pd.to_datetime(data['YEARMODA'], format='%Y%m%d', errors='ignore')
    data['YEAR'] = pd.DatetimeIndex(data['YEARMODA']).year
    data['MONTH'] = pd.DatetimeIndex(data['YEARMODA']).month
    data['DAY'] = pd.DatetimeIndex(data['YEARMODA']).day
    return data

# Number of years to consider
num_of_years = 20
year_files = os.listdir("../Data/archive/gsod_all_years")
year_files.sort()
year_files = year_files[-num_of_years:]

station_location = pd.read_csv('../Data/archive/isd-history.csv')
station_location = station_location.replace([0.0, -999.0, -999.9], np.nan)
station_location = station_location[pd.notnull(station_location['LAT']) & pd.notnull(station_location['LON'])]

# Filtering station location data based on year ranges
station_location = station_location[
    [int(re.findall('^\d{4}', str(end_year))[0]) == max([int(re.findall('\d+', year_file)[0]) for year_file in year_files])
     for end_year in station_location['END']]
]
station_location = station_location[
    [int(re.findall('^\d{4}', str(beg_year))[0]) <= min([int(re.findall('\d+', year_file)[0]) for year_file in year_files])
     for beg_year in station_location['BEGIN']]
]

# Generating labels for station locations
station_location['LBL'] = station_location[['STATION NAME', 'STATE', 'CTRY']].apply(lambda x: x.str.cat(sep=', '), axis=1)
station_location['ELEV_LBL'] = station_location['ELEV(M)'].apply(lambda x: 'Elevation: ' + str(x) + ' m' if ~np.isnan(x) else np.nan)
station_location['LBL'] = station_location[['LBL', 'ELEV_LBL']].apply(lambda x: x.str.cat(sep='<br>'), axis=1)
station_location = station_location.drop(['STATION NAME', 'ELEV_LBL', 'ICAO', 'BEGIN', 'END'], axis=1)

df_list = []
df_day_list = []

# Loop through each year file
for year_file in year_files:
    print(year_file)
    i = 0
    tar = tarfile.open("../Data/archive/gsod_all_years/" + year_file, "r")
    print(len(tar.getmembers()[1:]))
    for member in tar.getmembers()[1:]:
        name_parts = re.sub("\.op\.gz$", "", re.sub("^\./", "", member.name)).split("-")
        usaf = name_parts[0]
        wban = int(name_parts[1])
        if station_location[(station_location['USAF'] == usaf) & (station_location['WBAN'] == wban)].shape[0] != 0:
            i += 1
            f = tar.extractfile(member)
            f = gzip.open(f, 'rb')
            content = [re.sub(b" +", b",", line).decode("utf-8").split(",") for line in f.readlines()]
            content = process_content(content)
            df_day_list.append(content[content['YEARMODA'] == content['YEARMODA'].max()])
            content = content.groupby(['USAF', 'WBAN', 'YEAR', 'MONTH']).agg('median').reset_index()
            df_list.append(content)
    tar.close()

df = pd.concat(df_list)
df_day = pd.concat(df_day_list)

df_loc = pd.merge(df, station_location, how='inner', on=['USAF', 'WBAN'])
df_loc.to_csv('../Data/csv/IntermediateOutput2.csv')


In [ ]:
# Filter the DataFrame to include points within the defined boundaries
washington_df = df_loc[
    (df_loc['CTRY'] == "US") & (df_loc['STATE'] == 'WA')
]

In [ ]:
washington_df.head()

In [ ]:
unique_values = washington_df['USAF'].unique()
print("No. of weather stations in Washington State is", len(unique_values))

In [ ]:
washington_df.to_csv('../Data/csv/WashingtonWeather.csv')